In [1]:
import sys
import os
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_meinhold')
sys.path.append('../../cofe-python-analysis-tools/utils_meinhold')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca')
sys.path.append('../../cofe-python-analysis-tools/utils_zonca')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca\\pointing')
sys.path.append('../../cofe-python-analysis-tools/utils_zonca/pointing')
#sys.path.append('D:\\software_git_repos\\greenpol')
sys.path.append('../')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\')
sys.path.append('../telescope_control')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\VtoT\\')
sys.path.append('../telescope_control/VtoT')
import realtime_gp as rt
import numpy as np
import datetime 
import h5py
import pandas as pd
from pointingtools import compute_parallactic_angle, altaz2ha 
from planets import getlocation, getpointing
import warnings
from astropy.coordinates import AltAz, Angle, EarthLocation, ICRS, SkyCoord, frame_transform_graph
from astropy import units as u
import ephem
import matplotlib.pyplot as plt
import time
import planets
import convert
from datetime import timedelta
from collections import defaultdict

import Tkinter,tkFileDialog

from Tkinter import *
import ttk

import pickle
import glob

import cPickle

%pylab

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [2]:
def get_pointing_files(filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='D://software_git_repos/greenpol/telescope_control/data_aquisition/pointing_data/',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    
    return filelist

In [50]:
def get_gpstime(filelist,startrev=None, stoprev=None,angles_in_ints=False,azel_era=3):
    
    hpointing=[]
    filelist.sort()
    for f in filelist:
        stats=os.stat(f)
        if stats.st_size<22144:
            print f,stats.st_size
        if stats.st_size >= 22144:
            h=h5py.File(f)
            hh=h['data']
            hpointing.append(hh[hh['gpstime']>=hh['gpstime'][0]])
            #print hpointinng
            h.close()

    hpointing = np.concatenate(hpointing)
    #cut out blank lines from unfilled files
    if startrev != None:
        hpointing=hpointing[hpointing['gpstime'] > startrev]
    if stoprev != None:
        hpointing=hpointing[hpointing['gpstime'] < stoprev]

    #hrevlist,inds=np.unique(hpointing['gpstime'],return_index=True)
    hrevlist = hpointing['gpstime']
    comptimes = hpointing['computer time']
    az = hpointing['az']
    azoffset = hpointing['az offset']

    #gpstime = correct_gpstime(hrevlist, comptimemeans)
    #iscan = np.where(pp['gpstime'] >= 212700000)[0]
    gpstime = hrevlist
    #elmeans = elmeans[iscan]
    #azmeans = azmeans[iscan]
    #gpstime = gpstime[iscan]

    return {'gpstime':gpstime, 'comptime': comptimes, 'az':az, 'az offset': azoffset}

In [4]:
files = defaultdict(dict)
for d in range(19,28):
    print 'date %dth' % d
    files[d] = get_pointing_files()

date 19th
date 20th
date 21th
date 22th
date 23th
date 24th
date 25th
date 26th
date 27th


In [51]:
az = defaultdict(dict)
azoffset = defaultdict(dict)

for d in range(19,28):
    print d
    pp = get_gpstime(files[d][:2])
    az[d] = pp['az'][0]
    azoffset[d] = pp['az offset'][0]

19
20
21
22
23
24
25
26
27


In [55]:
for d in range(19,28):
    print 'day: %d, az: %f, offset: %f, enocder value: %f' % (d, az[d], azoffset[d], (az[d] - azoffset[d])%360.)

day: 19, az: 65.307180, offset: 49.179250, enocder value: 16.127930
day: 20, az: 64.126149, offset: 49.179250, enocder value: 14.946899
day: 21, az: 306.396657, offset: 49.179250, enocder value: 257.217407
day: 22, az: 286.357544, offset: 53.969238, enocder value: 232.388306
day: 23, az: 18.747616, offset: 178.587704, enocder value: 200.159912
day: 24, az: 254.085205, offset: 53.969238, enocder value: 200.115967
day: 25, az: 205.774297, offset: 242.188482, enocder value: 323.585815
day: 26, az: 126.370611, offset: 242.188482, enocder value: 244.182129
day: 27, az: 107.479619, offset: 242.188482, enocder value: 225.291138


In [56]:
gpst = defaultdict(dict)
cpt = defaultdict(dict)
az = defaultdict(dict)
gps = []
comp = []
azl = []

for d in range(19,28):
    print 'date %dth' % d
    pp = get_gpstime(files[d])
    gpst[d] = pp['gpstime']
    gps.append(gpst[d])
    cpt[d] = pp['comptime']
    comp.append(cpt[d])
    az[d] = pp['az'] - pp['az offset']
    azl.append(az[d])

date 19th
D:/software_git_repos/greenpol/telescope_control/data_aquisition/pointing_data/07-19-2018/21-15-00.h5 800
date 20th
date 21th
date 22th
date 23th
date 24th
date 25th
date 26th
date 27th


In [58]:
gpstime = np.concatenate(gps)
comptime = np.concatenate(comp)
aztot = np.concatenate(azl)

In [7]:
def sec_since_sunday(t):

    utcday = t 

    #days since last sunday
    idx = (utcday.weekday() + 1) % 7

    #date of previous sunday in universal coord
    sunday = utcday - datetime.timedelta(idx) 

    syear = int(str(sunday)[:4])
    smonth = int(str(sunday)[5:7])
    sday = int(str(sunday)[8:10])

    sunday = datetime.datetime(syear, smonth, sday, 0, 0, 0)

    #seconds since last sunday
    sundaysec = (utcday - sunday).total_seconds()

    return sundaysec

In [8]:
lt = defaultdict(dict)
lintime = []
stime = defaultdict(dict)
etime = defaultdict(dict)
for d in range(19,28):
    starttime = datetime.datetime.fromtimestamp(cpt[d][0])
    endtime = datetime.datetime.fromtimestamp(cpt[d][-1])
    for i in range(len(cpt[d])):
        if str(starttime)[8:10] != str(d) or str(endtime)[8:10] != str(d):
            starttime = datetime.datetime.fromtimestamp(cpt[d][i])
            endtime = datetime.datetime.fromtimestamp(cpt[d][-i-1])
        else:
            break
    st = sec_since_sunday(starttime)*1000
    et = sec_since_sunday(endtime)*1000
    stime[d] = starttime
    etime[d] = endtime
    lt[d] = np.linspace(st,et,len(gpst[d]))
    lintime.append(lt[d])
lintime = np.concatenate(lintime)

In [60]:
for d in range(19,29):
    figure()
    
    if d != 28:
        plot(az[d] % 360, label='gpstime')
        #plot(lt[d], label='linear time')
        title('date: %s, ms %d' % (str(stime[d]), lt[d][0]))
        legend()
    else:
        plot(aztot, label='gpstime')
        #plot(lintime, label='linear time')
        title('date: 07-19-2018 to 07-27-2018')
        legend()
        
    xlabel('samples')
    ylabel('gpstime (ms)')

In [66]:
print (15 + (33./60.))*1000*3600

55980000.0


In [10]:
def fixjumps(times):
    
    tdiff = np.diff(times)
    jumps = np.where(abs(tdiff) > 2**24/2)[0]
    
    for j in range(0, len(jumps[d])):
        if (correcttime[jumps[j]] - correcttime[jumps[j]+1])<0:
            if j == 0:
                correcttime[:jumps[j]+1] = correcttime[:jumps[j]+1] + (correcttime[jumps[j]+1]-correcttime[jumps[j]])
            else:
                correcttime[jumps[j-1]+1:jumps[j]+1] = correcttime[jumps[j-1]+1:jumps[j]+1] + (correcttime[jumps[j]+1] - correcttime[jumps[j]])
    
    return correctime

In [11]:
jumps = defaultdict(dict)
for d in range(19,28):
    tdiff = np.diff(gpst[d])
    ijump = np.where(abs(tdiff) > 2**24/2)[0]
    jumps[d] = ijump

In [12]:
correcttime = defaultdict(dict)

In [13]:
d = 25
print jumps[d]
print gpst[d][jumps[d]]

[  9180 181978]
[201325807  48656915]


In [14]:
d = 25
correcttime[d] = gpst[d].copy()
correcttime[d][jumps[d][0]+1:jumps[d][-1]+1] = correcttime[d][jumps[d][0]+1:jumps[d][-1]+1] + (correcttime[d][jumps[d][-1]+1] - correcttime[d][jumps[d][-1]])

In [16]:
figure()
plot(correcttime[d])
plot(lt[d], 'g-')

In [15]:
d = 24
print jumps[d]
print gpst[d][jumps[d]]

[28470]
[201326477]


In [16]:
correcttime[d] = gpst[d].copy()
correcttime[d][28471:] = correcttime[d][28471:] + (correcttime[d][28470] - correcttime[d][28471])

In [19]:
figure()
d=24
plot(correcttime[d])
plot(lt[d], 'g-')

In [17]:
correcttime[25][:9181] = correcttime[25][:9181] + (correcttime[24][-1] - correcttime[25][0])
#gpstc[:9181] = gpstc[:9181] + (gpstc[9181]-gpstc[9180])

In [18]:
d = 23
print jumps[d]
print gpst[d][jumps[d]]

[29787 29792 30534 31187 32058 32059 32061 32062 32101 32102 32127 32132
 32135 32136 32137 32139 32140 34717 35190 35200 35201 35202 39170]
[ 49040177 116154009  49786169 117549977  51333121 118442981  51336105
 118445959  51375847 118485707  51401963 118515785  51409889 118519763
  51411903 118522735  51414857 121109913  54530443 121649249  54541373
 121651231  58504137]


In [19]:
d = 23
correcttime[d] = gpst[d].copy()

In [20]:
for j in range(0, len(jumps[d])):
    if (correcttime[d][jumps[d][j]] - correcttime[d][jumps[d][j]+1])<0:
        if j == 0:
            correcttime[d][:jumps[d][j]+1] = correcttime[d][:jumps[d][j]+1] + (correcttime[d][jumps[d][j]+1]-correcttime[d][jumps[d][j]])
        else:
            correcttime[d][jumps[d][j-1]+1:jumps[d][j]+1] = correcttime[d][jumps[d][j-1]+1:jumps[d][j]+1] + (correcttime[d][jumps[d][j]+1] - correcttime[d][jumps[d][j]])

In [24]:
figure()
d=23
plot(correcttime[d])
plot(lt[d], 'g-')

In [21]:
d = 22
correcttime[d] = gpst[d].copy()

In [22]:
d = 21
correcttime[d] = gpst[d].copy()

In [23]:
print jumps[d]

[ 581442 1192036 1914507 1916249 1916275 1918014 1918040 1919779 1919805
 1921548 1921573 1923308 1923333 1925062 1925087 1926821 1926846 1928578
 1928603 1930330 1930355 1932089 1932114 1933842 1933867 1935599 1935624
 1937342]


In [24]:
for j in range(0, len(jumps[d])):
    if (correcttime[d][jumps[d][j]] - correcttime[d][jumps[d][j]+1])<0:
        if j == 0:
            correcttime[d][:jumps[d][j]+1] = correcttime[d][:jumps[d][j]+1] + (correcttime[d][jumps[d][j]+1]-correcttime[d][jumps[d][j]])
        else:
            correcttime[d][jumps[d][j-1]+1:jumps[d][j]+1] = correcttime[d][jumps[d][j-1]+1:jumps[d][j]+1] + (correcttime[d][jumps[d][j]+1] - correcttime[d][jumps[d][j]])

In [29]:
figure()
d = 21
plot(correcttime[d])
plot(lt[d], 'g-')

In [25]:
d = 20
correcttime[d] = gpst[d].copy()
#correcttime[d][:inds[-1]] = correcttime[d][:inds[-1]] + (correcttime[d+1][0] - correcttime[d][inds[-1]])

In [26]:
correcttime[d] += 2**28

In [32]:
figure()
d = 20
plot(correcttime[d])
plot(lt[d], 'g-')

In [27]:
gpstu,inds=np.unique(gpst[d],return_index=True)

In [34]:
figure()
d = 20
plot(correcttime[d][:inds[-1]])
plot(gpstu, 'k-')
plot(lt[d], 'g-')

In [28]:
correcttime[d][:inds[-1]] = correcttime[d][:inds[-1]] + (correcttime[d+1][0] - correcttime[d][inds[-1]])

In [29]:
print (correcttime[d+1][0] - correcttime[d][inds[-1]])

48649374


In [30]:
d = 19
correcttime[d] = gpst[d].copy()

In [39]:
figure()
d = 19
plot(correcttime[d])
plot(lt[d], 'g-')

In [31]:
d = 26
correcttime[d] = gpst[d].copy()
d = 27
correcttime[d] = gpst[d].copy()

In [55]:
correcttimes = []
for d in range(21,28):
    correcttimes.append(correcttime[d])
    
ctime = np.concatenate(correcttimes)

In [56]:
figure()
plot(ctime)

In [43]:
for d in range(21,29):
    figure()
    print d, len(correcttime[d])
    if d != 28:
        plot(correcttime[d], label='gpstime')
        plot(lt[d], label='linear time')
        title('date: %s, ms %d' % (str(stime[d]), lt[d][0]))
        legend()
    else:
        plot(ctime, label='gpstime')
        plot(lintime, label='linear time')
        title('date: 07-19-2018 to 07-27-2018')
        legend()
        
    xlabel('samples')
    ylabel('gpstime (ms)')

21 1937368
22 1807201
23 83029
24 1143126
25 373387
26 134905
27 41686
28 0


In [32]:
def get_h5_pointing(f,startrev=None, stoprev=None,angles_in_ints=False,azel_era=3):
    """
        modify to fast version, dont' average the multiple values, just take the first one. will cause a little bias but its very fast.
    also implmented removal of erroneous endof h5 file crap, and az outliers
    azel_era determines what az/el offsets to use, 1 means before 9/26/2013, 2 means 9/26-10/3 (inclusive), 3 means after 10/4.
    """

    #filelist is complete path list to the h5 files
    elconversion=-360./40000.
    azconversion=-360./(2.**16)
    #eloffset=8.2-3.17382
    #eloffset=5.026 # correction from 2013/08/02 moon crossing for both az and el this for ch 3! 
    #azoffset=4.41496
    # on 2013/08/20, updated converter.py to include these offsets, in theory they should now be zero.
    eloffset =0.0
    azoffset=0.0
    #ch1 would be azoffset=198.7810-187.663   or 11.118
    #             eloffset= 8.2-5.69175  
    #             eloffset= 2.5082
    #note October 4, 2013. Just found that data from 9/26/13 through 10/04/13 used old converter.py. So special 
    #offsets needed:
    #update, 10/8/13. ran get_cofe_crossing on second sun crossing from 10/04 (after subtracting 
    #template estimate of satellites,important effect). delta offsets: az=2.10,el=0.934
    #these then would be future offsets
    #update, 11/19/13. Ran mapmaking on lots of days, found an offset in the two daily crossings of the Crab- used ephem
    #to find the crossings, found that should get a self-consistent solution for both if the azimth is reduced by 4.51 degrees
    #and the elevation is increased by  2.241 degrees.
    '''
    if azel_era==1:
        eloffset=5.026
        azoffset=4.41496+140.0
    if azel_era==2:
        eloffset=5.026+0.934
        azoffset=4.41496+140.0+2.1
    if azel_era==3:
        eloffset=0.934+2.241
        azoffset=2.10 -4.51
    '''


    h=h5py.File(f)
    hh=h['data']
    hpointing = (hh[hh['gpstime']>=hh['gpstime'][0]])
    #print hpointinng
    h.close()

    #hpointing = np.concatenate(hpointing)
    #cut out blank lines from unfilled files
    if startrev != None:
        hpointing=hpointing[hpointing['gpstime'] > startrev]
    if stoprev != None:
        hpointing=hpointing[hpointing['gpstime'] < stoprev]

    hrevlist = hpointing['gpstime']#,inds=np.unique(hpointing['gpstime'],return_index=True)
    elmeans=hpointing['el']
    azmeans=hpointing['az']
    h1means = hpointing['H1']
    h2means = hpointing['H2']
    h3means = hpointing['H3']
    cryo1means = hpointing['Backend TSS']
    cryo2means = hpointing['Amplifier']
    cryo3means = hpointing['Cooler']
    cryo4means = hpointing['Calibrator']
    azlevmeans = hpointing['x tilt']
    ellevmeans = hpointing['y tilt']
    azoffsetmeans = hpointing['az offset']
    eloffsetmeans = hpointing['el offset']
    comptimemeans = hpointing['computer time']
    flagmeans = hpointing['flag']
    #("az offset", np.float), ("el offset", np.float), ("computer time", np.float), ("flag", np.int)])


    return {'el':elmeans,'az':azmeans,'gpstime':hrevlist, 'H1':h1means
            , 'H2':h2means, 'H3':h3means, 'Backend TSS':cryo1means
        , 'Amplifier':cryo2means, 'Cooler':cryo3means, 'Calibrator':cryo4means
        , 'x tilt':azlevmeans, 'y tilt':ellevmeans, 'az offset':azoffsetmeans, 'el offset':eloffsetmeans, 'computer time':comptimemeans
        , 'flag':flagmeans}

In [33]:
def fileStruct(n_array, date, time):

    os.chdir("D:/software_git_repos/greenpol/telescope_control/data_aquisition/pointing_data_corrected")	
    #t=ftime
    #date = date
    #time = time
    if not os.path.exists(date):#this is the first file being created for that time
        os.makedirs(date)
        #set index to 0


    path = '/'.join((date,time))
    path = '.'.join((path,"h5"))
    with h5py.File(str(path).replace("pkl","h5"), mode="w") as f:
        f.create_dataset("data", data=n_array.to_records(index=False))

In [34]:
badtimes = [21, 23, 24, 25]

In [35]:
correcttimes = []
for d in badtimes:
    correcttimes.append(correcttime[d])
    
ctime = np.concatenate(correcttimes)

In [36]:
flp = []
for d in badtimes:
    flp.append(files[d])
    
flp = np.concatenate(flp)

In [37]:
print flp[0]
print flp[0][-22:-12]
print flp[0][-11:-3]

D:/software_git_repos/greenpol/telescope_control/data_aquisition/pointing_data/07-21-2018/00-00-32.h5
07-21-2018
00-00-32


In [38]:
count = 0
for f in flp:
    pp = get_h5_pointing(f)
    for p in range(len(pp['gpstime'])):
        pp['gpstime'][p] = ctime[count]
        count += 1
    
    h5data=pd.DataFrame({"az": pp['az']})
    h5data["el"] = pp['el']
    h5data["gpstime"] = pp['gpstime']
    h5data["H1"] = pp['H1']
    h5data["H2"] = pp['H2']    
    h5data["H3"] = pp['H3']
    h5data["Backend TSS"] = pp['Backend TSS']
    h5data["Amplifier"] = pp['Amplifier']
    h5data["Cooler"] = pp['Cooler']
    h5data["Calibrator"] = pp['Calibrator']
    h5data["x tilt"] = pp['x tilt']
    h5data["y tilt"] = pp['y tilt']
    h5data["az offset"] = pp['az offset']
    h5data["el offset"] = pp['el offset']
    h5data["computer time"] = pp['computer time']
    h5data["flag"] = pp['flag']

    d = f[-22:-12]
    t = f[-11:-3]
    fileStruct(h5data, d, t)

In [54]:
figure()
plot(correcttime[24])

In [52]:
figure()
plot(gpst[24])